In [1]:
import os
import pandas
import urllib2
import requests
import bs4

In [2]:
url = 'http://www.footballlocks.com/nfl_point_spreads.shtml'
r  = requests.get(url)
data = r.text

soup = bs4.BeautifulSoup(data)

# get the tables w/ the spreads
tt = soup.findAll("table", {"width" : "580"})

/Users/amit/.virtualenvs/datascience/lib/python2.7/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 162 of the file /System/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


In [13]:
df_spreads = pandas.DataFrame()
for i in range(2):  # this probably needs to change once the preseason is over
    dfs = pandas.read_html(str(tt[i]), )
    df_spreads = df_spreads.append(dfs[0])

df_spreads.index = range(len(df_spreads))
df_spreads.columns = ['date', 'favorite', 'spread', 'underdog']    
filter_bad = df_spreads.favorite == 'Favorite'

df_spreads = df_spreads[~filter_bad] 

# get the home favorite
df_spreads['home_favorite'] = df_spreads.favorite.str.contains('^At ')
# fix any spreads that are tied (PK)
df_spreads.loc[df_spreads.spread.astype(unicode).str.contains('PK'), 'spread'] = .1

# flip sign on spread for away favorite
df_spreads['factor'] = 1
df_spreads.loc[df_spreads.home_favorite == True,'factor'] = -1
df_spreads['spreads2'] = df_spreads.spread.astype(float) * df_spreads.factor

# get the home team
df_spreads['home_team'] = df_spreads.favorite
home_filter = df_spreads.underdog.str.contains('^At ')
df_spreads.loc[home_filter, 'home_team'] = df_spreads.loc[home_filter, 'underdog']
df_spreads.home_team = df_spreads.home_team.str.replace('^At ', '')
df_spreads['datetime'] = pandas.to_datetime('2016/'+df_spreads.date, format='%Y/%m/%d %H:%M ET').dt.date
df_spreads

,date,favorite,spread,underdog,home_favorite,factor,spreads2,home_team,datetime
1,9/8 8:30 ET,Carolina,-3,At Denver,False,1,-3.0,Denver,2016-09-08
2,9/11 1:00 ET,At Atlanta,-3,Tampa Bay,True,-1,3.0,Atlanta,2016-09-11
3,9/11 1:00 ET,Minnesota,-2,At Tennessee,False,1,-2.0,Tennessee,2016-09-11
4,9/11 1:00 ET,At Philadelphia,-4,Cleveland,True,-1,4.0,Philadelphia,2016-09-11
5,9/11 1:00 ET,Cincinnati,-2.5,At NY Jets,False,1,-2.5,NY Jets,2016-09-11
6,9/11 1:00 ET,At New Orleans,-1,Oakland,True,-1,1.0,New Orleans,2016-09-11
7,9/11 1:00 ET,At Kansas City,-7,San Diego,True,-1,7.0,Kansas City,2016-09-11
8,9/11 1:00 ET,At Baltimore,-3,Buffalo,True,-1,3.0,Baltimore,2016-09-11
9,9/11 1:00 ET,At Houston,-6.5,Chicago,True,-1,6.5,Houston,2016-09-11
10,9/11 1:00 ET,Green Bay,-5.5,At Jacksonville,False,1,-5.5,Jacksonville,2016-09-11


In [14]:
# read in the master lines file
codeDir = "".join([os.environ['MLNFL_ROOT'], os.path.sep])
path_to_lines = codeDir + "data/lines/"
lines_file = path_to_lines + "nflAllLines.csv"

df_lines = pandas.read_csv(lines_file)
df_lines.Date = pandas.to_datetime(df_lines.Date)
df_lines

,Date,Visitor,Visitor Score,Home Team,Home Score,Line,Total Line,season,finalLine,yahooLine
0,1978-09-02,New York Giants,19.0,Tampa Bay Buccaneers,13.0,2.0,,1978,NaN,NaN
1,1978-09-03,Green Bay Packers,13.0,Detroit Lions,7.0,8.0,,1978,NaN,NaN
2,1978-09-03,Houston Oilers,14.0,Atlanta Falcons,20.0,-3.5,,1978,NaN,NaN
3,1978-09-03,Kansas City Chiefs,24.0,Cincinnati Bengals,23.0,8.0,,1978,NaN,NaN
4,1978-09-03,Miami Dolphins,20.0,New York Jets,33.0,-4.0,,1978,NaN,NaN
5,1978-09-03,Oakland Raiders,6.0,Denver Broncos,14.0,-4.0,,1978,NaN,NaN
6,1978-09-03,Pittsburgh Steelers,28.0,Buffalo Bills,17.0,-10.5,,1978,NaN,NaN
7,1978-09-03,Los Angeles Rams,16.0,Philadelphia Eagles,14.0,-6.0,,1978,NaN,NaN
8,1978-09-03,San Diego Chargers,24.0,Seattle Seahawks,20.0,-6.0,,1978,NaN,NaN
9,1978-09-03,San Francisco 49ers,7.0,Cleveland Browns,24.0,4.5,,1978,NaN,NaN


In [17]:
# find the right week/game and update the spread
week_filter = (df_lines.Date <= df_spreads.datetime.max()) & (df_lines.Date >= df_spreads.datetime.min())

for ii,rr in df_spreads.iterrows():
    print ii, rr['home_team'], rr['spreads2']
    if 'NY' in rr['home_team']:
        rr['home_team'] = rr['home_team'].split(' ')[1]
    game_filter = df_lines[week_filter]['Home Team'].str.contains(rr['home_team'])
    irow = df_lines[week_filter][game_filter].index[0]
    df_lines.loc[irow, 'Line'] = rr['spreads2']

1 Denver -3.0
2 Atlanta 3.0
3 Tennessee -2.0
4 Philadelphia 4.0
5 NY Jets -2.5
6 New Orleans 1.0
7 Kansas City 7.0
8 Baltimore 3.0
9 Houston 6.5
10 Jacksonville -5.5
11 Seattle 10.5
12 Dallas -1.0
13 Indianapolis 3.5
14 Arizona 6.0
15 Washington -3.0
16 San Francisco -2.5


In [18]:
lines_file2 = lines_file = path_to_lines + "nflAllLines2.csv"
df_lines.to_csv(lines_file, index=False)